Display a bunch of info about a bill in one place

- Cosponsors (map)
- Committee members (map)
- Current status
- Scheduled hearings

Later: Combine info for House & Senate versions?

In [1]:
bill_number = "S459"
bill_session = 192

In [2]:
import requests
import json
from datetime import datetime, timezone
from IPython import display
from tqdm.notebook import tqdm

In [3]:
def get_json(url, verbose=True):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

## Get the bill details

<https://malegislature.gov/api/swagger/index.html?url=/api/swagger/v1/swagger.json#/Documents>

In [4]:
document = get_json(f"https://malegislature.gov/api/GeneralCourts/{bill_session}/Documents/{bill_number}")

In [5]:
def transform_bill(bill):
    transformed_bill = {
        k: v for k, v in bill.items()
        if k not in {"Pinslip", "DocumentText", "EmergencyPreamble"}
    }
    transformed_bill["Updated"] = datetime.now(timezone.utc).isoformat()
    return transformed_bill

bill = transform_bill(document)
display.JSON(bill)

<IPython.core.display.JSON object>

## Get the committee(s) and members

<https://malegislature.gov/api/swagger/index.html?url=/api/swagger/v1/swagger.json#/Committees>  
via [Committeees notebook](Committees.ipynb)

In [6]:
with open("ma_committees.json") as f:
    committees = json.load(f)
    
print(len(committees))

55


Members aren't included in the committee data, so we need to find all members where the current committee is in their list of committees.

<https://malegislature.gov/api/swagger/index.html?url=/api/swagger/v1/swagger.json#/LegislativeMembers>  
via [LegislativeMembers notebook](LegislativeMembers.ipynb)

In [7]:
with open("ma_legislative_members.json") as f:
    members = json.load(f)
len(members)

200

Member summary follows the pattern of other API responses, where the list of members contains only what's needed to get more details from the API or on the website.

In [8]:
def summarize_member(member):
    return {
        'GeneralCourtNumber': member['GeneralCourtNumber'],
        'MemberCode': member['MemberCode'],
        'Details': f"http://malegislature.gov/api/GeneralCourts/{member['GeneralCourtNumber']}/LegislativeMembers/{member['MemberCode']}"
    }

def get_committee_members(committee):
    committee_members = [
        m for m in members
        if committee['CommitteeCode'] in {c['CommitteeCode'] for c in m['Committees']}
    ]

    return {
        f"{branch}Members": [
            summarize_member(m) for m in committee_members
            if m['Branch'] == branch
        ]
        for branch in ["House", "Senate"]
    }

def update_committee(committee):
    committee = {
        k: v for k, v in committee.items()
        if k not in {"DocumentsBeforeCommittee", "ReportedOutDocuments"}
    }
    committee.update(get_committee_members(committee))
    return committee

bill_committees = {
    document_status.replace("Documents", ""): [
        update_committee(c) for c in committees
        if bill_number in {d["BillNumber"] for d in c[document_status]}
    ]
    for document_status in {"DocumentsBeforeCommittee", "ReportedOutDocuments"}
}

assert len(bill_committees['BeforeCommittee']) <= 1
display.JSON(bill_committees)

<IPython.core.display.JSON object>

In [9]:
try:
    bill["BeforeCommittee"] = bill_committees["BeforeCommittee"][0]
except IndexError:
    bill["BeforeCommittee"] = {}

bill["ReportedOut"] = bill_committees["ReportedOut"]

display.JSON(bill)

<IPython.core.display.JSON object>

## TODO

Add details for:

- `BillHistory`
- `Amendments`
- `RollCalls`
- `BeforeCommitee.Hearings`

## Save bill details

In [10]:
filename = f"../dist/ma_bill_{bill['GeneralCourtNumber']}_{bill['BillNumber']}.json"
with open(filename, "w") as f:
    json.dump(bill, f, indent=2)
!ls -lh {filename}

-rw-r--r--  1 bhrutledge  staff    11K Jul 17 13:07 ../dist/ma_bill_192_S459.json
